In [4]:
import requests
from bs4 import BeautifulSoup
import re
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import html5lib
path = r'C:\Program Files\Chrome_Driver/chromedriver.exe'

In [79]:
from datetime import datetime, timedelta

In [25]:
#-----------------------------------------  Second Page  ----------------------------------------------------------

class IronhackSpider:
    
    def __init__(self, url_pattern, pages_to_scrape=2, sleep_interval=-1, content_parser=None,selenium=True,driver=None,main_table=None):
        self.url_pattern = url_pattern
        self.pages_to_scrape = pages_to_scrape
        self.sleep_interval = sleep_interval
        self.content_parser = content_parser
        self.selenium = selenium
        self.main_table = main_table
        self.driver = driver

#[2]     
    def scrape_url(self, url,selenium):
        if selenium == True:
            response_s_p = self.selenium_second_page(url)
            result = self.content_parser(response_s_p)
            result['Id_second_page'] = [re.findall('[0-9].*$',url)[0] for l in range(len(result))]
        else:
            try:
                response = requests.get(url)
                if response.status_code < 300:
                    print('Request was successful')
                elif response.status_code >= 400 and r.status_code < 500:
                    print('Request failed because the resource either does not exist or is forbidden')
                else:
                    print('Request failed because the response server encountered an error')
            except requests.exceptions.Timeout:
                print("It is taking too long, timeout error!")
            except requests.exceptions.TooManyRedirects:
                print("There was too many redirections, the url exceeded maximum redirects")
            except requests.exceptions.SSLError:
                print("SSL not working, SSL Certificate Error")
            except requests.exceptions.RequestException as e:
                print("Connection to url is not working, try again later!")
            result = self.content_parser(response.content)
        self.output_results(result)
          
#[4]
    def output_results(self, r):
        self.main_table = self.main_table.append(r)
    
        
#[1]
    def kickstart(self):
        for i in range(1, self.pages_to_scrape+1):
            self.scrape_url(self.url_pattern,self.selenium)
            if self.sleep_interval > 0:
                time.sleep(self.sleep_interval)
                # aqui if para definir si el url es una lista o no
        return self.main_table
    
    def selenium_second_page(self,url_crawl):
            
            self.driver.get(url_crawl)
            time.sleep(3)
            page_source = self.driver.page_source
            return page_source
            
#[3]
def parser_second_page(content):    
    soup = BeautifulSoup(content,'html')
    table = soup.find_all('table')
    df = pd.read_html(str(table))[0]
    table = soup.find( "table")
    links=[row['href'] for row in table.findAll("a")]
    df['Links'] = links
    return df
    
    
    


#-----------------------------------------  Main Page  ----------------------------------------------------------


# Class para scrapear main page y pagians dentro
class main_page (IronhackSpider):
    def __init__(self,url_crawl,url_pattern, pages_to_scrape=10, sleep_interval=-1, content_parser=None,selenium=True):
        self.url_crawl = url_crawl
        self.content_parser = content_parser
        self.main_table = pd.DataFrame()

        super().__init__(url_pattern, pages_to_scrape=3, sleep_interval=-1, content_parser=main_page_parser,selenium=True,main_table=pd.DataFrame())
        
#[2]
    def scrape_main_page(self,url_crawl,selenium=True):
        if selenium:
            response,driver = selenium_main_page(url_crawl)
            id_main, evento, lugar, fecha, suf_links_txt = self.content_parser(response)
        else:
            response = requests.get(url_crawl, timeout=10)
            id_main, evento, lugar, fecha, suf_links_txt = self.content_parser(response)
            
        link_completo = [str(url_crawl)+str(l) for l in suf_links_txt]
        df = pd.DataFrame(list(zip(id_main,lugar,fecha,evento,link_completo)),columns =['Id_main_page','Place', 'Date','Event','Link'])
        saving_file(df)
        
        # Aqui lama al segundo scrape
        for n,pages in enumerate(link_completo):
        # odigo para el ambiente de prueba
            #if n> self.pages_to_scrape:              
             #   driver.quit()
              #  break
            second_page = IronhackSpider(pages, pages_to_scrape=1, content_parser=parser_second_page, selenium=True,driver=driver,main_table=self.main_table) #len(link_completo)
            self.main_table = second_page.kickstart()
        driver.quit()
        return
        
#[1]
    def kickstart_main_page(self):       #aqui se puede cambiar entre selenium o scrape
        self.scrape_main_page(self.url_crawl,self.selenium)
        
def saving_file(df):
        df.to_csv('Main_Page_Scrap.csv')
        
#[3]        
    # returns lists with place,date,event,suf_links
def main_page_parser(content_html):
        soup = BeautifulSoup(content_html,'html')
        links = soup.select('#custom-firebase-previous a')
        suf_links_txt = [re.search('show-info[^"]*',str(i)).group(0) for i in links]
        nombre = soup.select('#custom-firebase-previous p') 
        info= [element.text.strip() for element in nombre]
        ev = [info[i] for i in range(0,len(info),2)]
        fecha = [info[i] for i in range(1,len(info),2)]
        lugar = []
        for w in ev:
            first, *middle, last = w.split()
            try:
                m = re.search(r'[A-Z]+[a-z]+$', last).group(0)
                lugar.append(m)
            except:
                m = last
                lugar.append(m)        
        evento = [e.replace(l,"") for l,e in zip(lugar,ev)]
        id_main = [re.findall('[0-9].*$',l)[0] for l in suf_links_txt]
        return id_main,evento,lugar,fecha,suf_links_txt
    
def selenium_main_page(url_crawl):
        driver = webdriver.Chrome(path)
        driver.get(url_to_crawl)
        time.sleep(3)
        page_source = driver.page_source
        #elem = driver.find_element_by_name('div') 
        return page_source,driver
    
url_to_crawl = 'https://ecuestre.digital/shows/'

info_shows = main_page(url_to_crawl, 2, content_parser=main_page_parser,selenium=True)
info_shows.kickstart_main_page()


https://ecuestre.digital/shows/show-info?id=714
     #                                               Name  \
0    A                                   HORARIOS DOMINGO   
1   30                                 Abierta - 0.85 Mts   
2   29               RELEVO - 0.95 Mts / RELEVO - 105 Mts   
3   28              PRUEBA POR EQUIPOS - Abierta 1.15 Mts   
4   27                                   Abierta 1.25 Mts   
5   26                                 Abierta - 1.35 Mts   
6   25                                   Elite - 1.45 Mts   
7   24                                          Lead Line   
8   23                       Verticales Menores y Mayores   
9   22                         Novicios Menores y Mayores   
10   A                                    HORARIOS SABADO   
11  21  Abierta - Juvenil - Caballos de 7 años - 1.30 Mts   
12  20               Caballos Jovenes - 6 años - 1.20 Mts   
13  19                  Owner Amateur y Abierta -1.20 Mts   
14  18                    Caballos se

     #                                               Name  \
0    A                                   HORARIOS DOMINGO   
1   30                                 Abierta - 0.85 Mts   
2   29               RELEVO - 0.95 Mts / RELEVO - 105 Mts   
3   28              PRUEBA POR EQUIPOS - Abierta 1.15 Mts   
4   27                                   Abierta 1.25 Mts   
..  ..                                                ...   
36   5  PRE-INFANTIL A)Leccion Children Individual B) ...   
37   4  JINETES DEBUTANTES A)Leccion Children Individu...   
38   3  CABALLOS DEBUTANTES A)Leccion Children Individ...   
39   2                                          JUVENILES   
40   1                                         SMALL TOUR   

                                        Article       Ring        Date  \
0                                         Speed  Principal  19-09-2021   
1                                  Tiempo Ideal  Principal  19-09-2021   
2                                         Spe

     #                                   Name                    Article  \
0    A                       HORARIOS DOMINGO                      Speed   
1   30                     Abierta - 0.85 Mts               Tiempo Ideal   
2   29   RELEVO - 0.95 Mts / RELEVO - 105 Mts                      Speed   
3   28  PRUEBA POR EQUIPOS - Abierta 1.15 Mts                      Speed   
4   27                       Abierta 1.25 Mts                   Jump Off   
..  ..                                    ...                        ...   
12   8                        1.00 CRONOMETRO                      Speed   
13   5                             LIBRE 1.30                      Speed   
14   4                             LIBRE 1.20                   Jump Off   
15   3                       TIEMPO IDEAL FEM  Tiempo Ideal 2 Recorridos   
16   1                               TRAINING                   Training   

         Ring        Date Unnamed: 5                       Links  \
0   Principal  19-0

        #                                       Name       Article  \
0       A                           HORARIOS DOMINGO         Speed   
1      30                         Abierta - 0.85 Mts  Tiempo Ideal   
2      29       RELEVO - 0.95 Mts / RELEVO - 105 Mts         Speed   
3      28      PRUEBA POR EQUIPOS - Abierta 1.15 Mts         Speed   
4      27                           Abierta 1.25 Mts      Jump Off   
..    ...                                        ...           ...   
11      8           Abierta 0.65m - Copa Blue Luxury  Tiempo Ideal   
12  4 y 5         Abierta 1.05m y Categoría Infantil         Speed   
13      3  Gran Premio Maspor Marine - 1.25m - Dia 1       Table C   
14      2                              Abierta 0.40m  Tiempo Ideal   
15      1                              Abierta 0.20m      Training   

               Ring        Date Unnamed: 5                       Links  \
0         Principal  19-09-2021  See Class  /results?id=16481&show=714   
1         P

     #                                   Name       Article       Ring  \
0    A                       HORARIOS DOMINGO         Speed  Principal   
1   30                     Abierta - 0.85 Mts  Tiempo Ideal  Principal   
2   29   RELEVO - 0.95 Mts / RELEVO - 105 Mts         Speed  Principal   
3   28  PRUEBA POR EQUIPOS - Abierta 1.15 Mts         Speed  Principal   
4   27                       Abierta 1.25 Mts      Jump Off  Principal   
..  ..                                    ...           ...        ...   
33  2B                             JD/ABIERTA         Speed      SALTO   
34  2A  1.05 JINETES DEBUTANTES/ PRE INFANTIL         Speed      SALTO   
35   2                    1.05 CAB NOVICIOS B         Speed      SALTO   
36   1        1.25 C7A/ JB / PRE JUV/ ABIERTA         Speed      SALTO   
37  00                   SOLICITUD DE CUADRAS      Training        NaN   

          Date Unnamed: 5                       Links Id_second_page  
0   19-09-2021  See Class  /results?id=1

...

     #                                      Name       Article       Ring  \
0    A                          HORARIOS DOMINGO         Speed  Principal   
1   30                        Abierta - 0.85 Mts  Tiempo Ideal  Principal   
2   29      RELEVO - 0.95 Mts / RELEVO - 105 Mts         Speed  Principal   
3   28     PRUEBA POR EQUIPOS - Abierta 1.15 Mts         Speed  Principal   
4   27                          Abierta 1.25 Mts      Jump Off  Principal   
..  ..                                       ...           ...        ...   
6    5              A-Jinetes, B-Jin.1a 1.10 mts         Speed      Verde   
7    4  A-Mayores, B-Menores, C-Jin.1a. 1.00 mts         Speed      Verde   
8    3             A-Mayores, B-Menores 0.90 mts  Tiempo Ideal      Verde   
9    2                             CN5A 1.10 mts  Tiempo Ideal      Verde   
10   1                        A-CN4, F/C 1.00 mt  Tiempo Ideal      Verde   

          Date Unnamed: 5                       Links Id_second_page  
0   

      #                                   Name       Article       Ring  \
0     A                       HORARIOS DOMINGO         Speed  Principal   
1    30                     Abierta - 0.85 Mts  Tiempo Ideal  Principal   
2    29   RELEVO - 0.95 Mts / RELEVO - 105 Mts         Speed  Principal   
3    28  PRUEBA POR EQUIPOS - Abierta 1.15 Mts         Speed  Principal   
4    27                       Abierta 1.25 Mts      Jump Off  Principal   
..  ...                                    ...           ...        ...   
36  5.0                       TRABAJO 1.30 MTS  Art. 238.1.1   PICADERO   
37  4.0                       TRABAJO 1.20 MTS  Art. 238.1.1   PICADERO   
38  3.0                       TRABAJO 1.10 MTS  Art. 238.1.1   PICADERO   
39  2.0                        TRABAJO 1.00 MT  Art. 238.1.1   PICADERO   
40  1.0                  TRABAJO 0.80-0.90 MTS  Art. 238.1.1   PICADERO   

          Date Unnamed: 5                       Links Id_second_page  
0   19-09-2021  See Class  /

     #                                   Name       Article       Ring  \
0    A                       HORARIOS DOMINGO         Speed  Principal   
1   30                     Abierta - 0.85 Mts  Tiempo Ideal  Principal   
2   29   RELEVO - 0.95 Mts / RELEVO - 105 Mts         Speed  Principal   
3   28  PRUEBA POR EQUIPOS - Abierta 1.15 Mts         Speed  Principal   
4   27                       Abierta 1.25 Mts      Jump Off  Principal   
..  ..                                    ...           ...        ...   
26   4                       Abierta 1.10 mts         Speed  Principal   
27   3                       Abierta 1.20 mts         Speed  Principal   
28   2                       Abierta 1.30 mts         Speed  Principal   
29   1                       Abierta 1.40 mts         Speed  Principal   
30   A                        Horario Viernes         Speed    Horario   

          Date Unnamed: 5                       Links Id_second_page  
0   19-09-2021  See Class  /results?id=1

     #                                   Name                        Article  \
0    A                       HORARIOS DOMINGO                          Speed   
1   30                     Abierta - 0.85 Mts                   Tiempo Ideal   
2   29   RELEVO - 0.95 Mts / RELEVO - 105 Mts                          Speed   
3   28  PRUEBA POR EQUIPOS - Abierta 1.15 Mts                          Speed   
4   27                       Abierta 1.25 Mts                       Jump Off   
..  ..                                    ...                            ...   
28   7      A( Abierta 1.10 M B( Libre 1.10 M                          Speed   
29   1      FEI Jumping Challenge Categoría C  Dos Recorridos Art. 273.3.3.2   
30   6                     Prenovicios 1.00 M                   Art. 238.1.1   
31   5      A( Abierta 1.00 M B( Libre 1.00 M                          Speed   
32   4      A( Abierta 0.90 M B( Libre 0.90 M                   Tiempo Ideal   

         Ring        Date Unnamed: 5   

     #                                   Name                        Article  \
0    A                       HORARIOS DOMINGO                          Speed   
1   30                     Abierta - 0.85 Mts                   Tiempo Ideal   
2   29   RELEVO - 0.95 Mts / RELEVO - 105 Mts                          Speed   
3   28  PRUEBA POR EQUIPOS - Abierta 1.15 Mts                          Speed   
4   27                       Abierta 1.25 Mts                       Jump Off   
..  ..                                    ...                            ...   
24   5          A)Abierta 1.10M B)Libre 1.10M                          Speed   
25   4              Fei Challenge Categoria C  Dos Recorridos Art. 273.3.3.2   
26   3                      Prenovicios 1.00M                   Art. 238.1.1   
27   2         A)Abierta 1.00M B) Libre 1.00M                          Speed   
28   1             A) Abierta / B) Libre 0.90                   Tiempo Ideal   

         Ring        Date Unnamed: 5   

In [97]:
#df = pd.read_csv('Main_Page_Scrape.csv')

## Data Cleaning

In [99]:
df['Fecha Final'] = [str(re.search('\s.*',i).group(0)).replace('-','').strip() for i in df['Fecha']]
df['Fecha Final'] = pd.to_datetime(df['Fecha Final'])
df

,Unnamed: 0,Id_main_page,Lugar,Fecha,Evento,Link,Fecha Inicio,Fecha Final
0,0,714,Guatemala,17/9/2021 - 19/9/2021,El Pulte,https://ecuestre.digital/shows/show-info?id=714,17/9/2021,19/9/2021
1,1,701,Ecuador,17/9/2021 - 19/9/2021,Club Hípico Yaguachi,https://ecuestre.digital/shows/show-info?id=701,17/9/2021,19/9/2021
2,2,712,Venezuela,17/9/2021 - 19/9/2021,Club Campestre Los Cortijos,https://ecuestre.digital/shows/show-info?id=712,17/9/2021,19/9/2021
3,3,660,MI,17/9/2021 - 19/9/2021,"Major League Show JumpingTraverse City,",https://ecuestre.digital/shows/show-info?id=660,17/9/2021,19/9/2021
4,4,716,Colombia,16/9/2021 - 19/9/2021,Club Campestre De Cali,https://ecuestre.digital/shows/show-info?id=716,16/9/2021,19/9/2021
5,5,709,Uruguay,11/9/2021 - 11/9/2021,Centro De Entrenamiento De Caballos Deportivos,https://ecuestre.digital/shows/show-info?id=709,11/9/2021,11/9/2021
6,6,710,California,10/9/2021 - 12/9/2021,Jockey Club De Caliente A.cBaja,https://ecuestre.digital/shows/show-info?id=710,10/9/2021,12/9/2021
7,7,704,Ecuador,10/9/2021 - 12/9/2021,Arrayanes Country Club,https://ecuestre.digital/shows/show-info?id=704,10/9/2021,12/9/2021
8,8,688,Guadalajara,9/9/2021 - 12/9/2021,"Hípica Gdl, A.c.",https://ecuestre.digital/shows/show-info?id=688,9/9/2021,12/9/2021
9,9,711,Monterrey,9/9/2021 - 12/9/2021,"Centro Ecuestre San Pedro, A.c.",https://ecuestre.digital/shows/show-info?id=711,9/9/2021,12/9/2021


In [100]:
df['Fecha Inicio'] = [str(re.search('.*\s',i).group(0)).replace('-','').strip() for i in df['Fecha']]
df['Fecha Inicio'] = pd.to_datetime(df['Fecha Inicio'])
df

,Unnamed: 0,Id_main_page,Lugar,Fecha,Evento,Link,Fecha Inicio,Fecha Final
0,0,714,Guatemala,17/9/2021 - 19/9/2021,El Pulte,https://ecuestre.digital/shows/show-info?id=714,17/9/2021,19/9/2021
1,1,701,Ecuador,17/9/2021 - 19/9/2021,Club Hípico Yaguachi,https://ecuestre.digital/shows/show-info?id=701,17/9/2021,19/9/2021
2,2,712,Venezuela,17/9/2021 - 19/9/2021,Club Campestre Los Cortijos,https://ecuestre.digital/shows/show-info?id=712,17/9/2021,19/9/2021
3,3,660,MI,17/9/2021 - 19/9/2021,"Major League Show JumpingTraverse City,",https://ecuestre.digital/shows/show-info?id=660,17/9/2021,19/9/2021
4,4,716,Colombia,16/9/2021 - 19/9/2021,Club Campestre De Cali,https://ecuestre.digital/shows/show-info?id=716,16/9/2021,19/9/2021
5,5,709,Uruguay,11/9/2021 - 11/9/2021,Centro De Entrenamiento De Caballos Deportivos,https://ecuestre.digital/shows/show-info?id=709,11/9/2021,11/9/2021
6,6,710,California,10/9/2021 - 12/9/2021,Jockey Club De Caliente A.cBaja,https://ecuestre.digital/shows/show-info?id=710,10/9/2021,12/9/2021
7,7,704,Ecuador,10/9/2021 - 12/9/2021,Arrayanes Country Club,https://ecuestre.digital/shows/show-info?id=704,10/9/2021,12/9/2021
8,8,688,Guadalajara,9/9/2021 - 12/9/2021,"Hípica Gdl, A.c.",https://ecuestre.digital/shows/show-info?id=688,9/9/2021,12/9/2021
9,9,711,Monterrey,9/9/2021 - 12/9/2021,"Centro Ecuestre San Pedro, A.c.",https://ecuestre.digital/shows/show-info?id=711,9/9/2021,12/9/2021


In [105]:
df['Duración'] = df['Fecha Final']-df['Fecha Inicio']
df

,Unnamed: 0,Id_main_page,Lugar,Fecha,Evento,Link,Fecha Inicio,Fecha Final,Duración
0,0,714,Guatemala,17/9/2021 - 19/9/2021,El Pulte,https://ecuestre.digital/shows/show-info?id=714,2021-09-17,2021-09-19,2 days
1,1,701,Ecuador,17/9/2021 - 19/9/2021,Club Hípico Yaguachi,https://ecuestre.digital/shows/show-info?id=701,2021-09-17,2021-09-19,2 days
2,2,712,Venezuela,17/9/2021 - 19/9/2021,Club Campestre Los Cortijos,https://ecuestre.digital/shows/show-info?id=712,2021-09-17,2021-09-19,2 days
3,3,660,MI,17/9/2021 - 19/9/2021,"Major League Show JumpingTraverse City,",https://ecuestre.digital/shows/show-info?id=660,2021-09-17,2021-09-19,2 days
4,4,716,Colombia,16/9/2021 - 19/9/2021,Club Campestre De Cali,https://ecuestre.digital/shows/show-info?id=716,2021-09-16,2021-09-19,3 days
5,5,709,Uruguay,11/9/2021 - 11/9/2021,Centro De Entrenamiento De Caballos Deportivos,https://ecuestre.digital/shows/show-info?id=709,2021-11-09,2021-11-09,0 days
6,6,710,California,10/9/2021 - 12/9/2021,Jockey Club De Caliente A.cBaja,https://ecuestre.digital/shows/show-info?id=710,2021-10-09,2021-12-09,61 days
7,7,704,Ecuador,10/9/2021 - 12/9/2021,Arrayanes Country Club,https://ecuestre.digital/shows/show-info?id=704,2021-10-09,2021-12-09,61 days
8,8,688,Guadalajara,9/9/2021 - 12/9/2021,"Hípica Gdl, A.c.",https://ecuestre.digital/shows/show-info?id=688,2021-09-09,2021-12-09,91 days
9,9,711,Monterrey,9/9/2021 - 12/9/2021,"Centro Ecuestre San Pedro, A.c.",https://ecuestre.digital/shows/show-info?id=711,2021-09-09,2021-12-09,91 days


In [122]:
#df.to_csv('Main_Page_Scrape.csv')

In [123]:
df_2 = pd.read_csv('Second_Page_Scrape.csv')
df_2

,Unnamed: 0,#,Name,Article,Ring,Date,Links,Links.1,Id_second_page
0,0,A,HORARIOS DOMINGO,Speed,Principal,19-09-2021,See Class,/results?id=16481&show=714,714
1,1,30,Abierta - 0.85 Mts,Tiempo Ideal,Principal,19-09-2021,See Class,/results?id=16478&show=714,714
2,2,29,RELEVO - 0.95 Mts / RELEVO - 105 Mts,Speed,Principal,19-09-2021,See Class,/results?id=16477&show=714,714
3,3,28,PRUEBA POR EQUIPOS - Abierta 1.15 Mts,Speed,Principal,19-09-2021,See Class,/results?id=16476&show=714,714
4,4,27,Abierta 1.25 Mts,Jump Off,Principal,19-09-2021,See Class,/results?id=16475&show=714,714
...,...,...,...,...,...,...,...,...,...
1319,23,7,Abierta 0.85,Speed,Principal,3/7/2021,See Class,/results?id=15059&show=650,650
1320,24,6,Abierta 0.80,Tiempo Ideal,Principal,3/7/2021,See Class,/results?id=15058&show=650,650
1321,25,5,Caballos de 1er Año,Tiempo Ideal,Principal,3/7/2021,See Class,/results?id=15057&show=650,650
1322,26,3 & 4,Verticales Menores y Mayores,Tiempo Ideal,Principal,3/7/2021,See Class,/results?id=15056&show=650,650


In [125]:
df_2.drop('Links',axis=1,inplace=True)
df_2

,Unnamed: 0,#,Name,Article,Ring,Date,Links.1,Id_second_page
0,0,A,HORARIOS DOMINGO,Speed,Principal,19-09-2021,/results?id=16481&show=714,714
1,1,30,Abierta - 0.85 Mts,Tiempo Ideal,Principal,19-09-2021,/results?id=16478&show=714,714
2,2,29,RELEVO - 0.95 Mts / RELEVO - 105 Mts,Speed,Principal,19-09-2021,/results?id=16477&show=714,714
3,3,28,PRUEBA POR EQUIPOS - Abierta 1.15 Mts,Speed,Principal,19-09-2021,/results?id=16476&show=714,714
4,4,27,Abierta 1.25 Mts,Jump Off,Principal,19-09-2021,/results?id=16475&show=714,714
...,...,...,...,...,...,...,...,...
1319,23,7,Abierta 0.85,Speed,Principal,3/7/2021,/results?id=15059&show=650,650
1320,24,6,Abierta 0.80,Tiempo Ideal,Principal,3/7/2021,/results?id=15058&show=650,650
1321,25,5,Caballos de 1er Año,Tiempo Ideal,Principal,3/7/2021,/results?id=15057&show=650,650
1322,26,3 & 4,Verticales Menores y Mayores,Tiempo Ideal,Principal,3/7/2021,/results?id=15056&show=650,650


In [132]:

df_2['Date'] = pd.to_datetime(df_2['Date'])
print(df_2['Date'])

pandas.core.series.Series

In [136]:
df_2['Id_article'] =  df_2['#'] + '-' + df_2['Article']
df_2

,Unnamed: 0,#,Name,Article,Ring,Date,Links.1,Id_second_page,Id_article
0,0,A,HORARIOS DOMINGO,Speed,Principal,2021-09-19,/results?id=16481&show=714,714,A-Speed
1,1,30,Abierta - 0.85 Mts,Tiempo Ideal,Principal,2021-09-19,/results?id=16478&show=714,714,30-Tiempo Ideal
2,2,29,RELEVO - 0.95 Mts / RELEVO - 105 Mts,Speed,Principal,2021-09-19,/results?id=16477&show=714,714,29-Speed
3,3,28,PRUEBA POR EQUIPOS - Abierta 1.15 Mts,Speed,Principal,2021-09-19,/results?id=16476&show=714,714,28-Speed
4,4,27,Abierta 1.25 Mts,Jump Off,Principal,2021-09-19,/results?id=16475&show=714,714,27-Jump Off
...,...,...,...,...,...,...,...,...,...
1319,23,7,Abierta 0.85,Speed,Principal,2021-03-07,/results?id=15059&show=650,650,7-Speed
1320,24,6,Abierta 0.80,Tiempo Ideal,Principal,2021-03-07,/results?id=15058&show=650,650,6-Tiempo Ideal
1321,25,5,Caballos de 1er Año,Tiempo Ideal,Principal,2021-03-07,/results?id=15057&show=650,650,5-Tiempo Ideal
1322,26,3 & 4,Verticales Menores y Mayores,Tiempo Ideal,Principal,2021-03-07,/results?id=15056&show=650,650,3 & 4-Tiempo Ideal


In [134]:
df_2.to_csv('Second_Page_Scrape.csv')

## Proof Enviroment

In [33]:
url_to_crawl = 'https://ecuestre.digital/shows/show-info?id=716'
driver = webdriver.Chrome(path)
driver.get(url_to_crawl)
time.sleep(3)
page_source = driver.page_source
#elem = driver.find_element_by_name('div')
print(page_source)
driver.quit()
soup = BeautifulSoup(page_source,'html')
print('\n',soup)

<html lang="es-MX" class=" js no-touch cssanimations csstransitions js csstransforms csstransforms3d csstransitions" style=""><head>
<meta charset="UTF-8">
<!--[if IE]><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><![endif]-->
<title>Ecuestre Digital :   Show Info</title>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport">
<meta content="" name="description">
<link rel="shortcut icon" href="https://ecuestre.digital/wp-content/uploads/2021/01/cropped-Logo-Cuadrado-520-e1611039350171.png">
<link rel="apple-touch-icon-precomposed" sizes="144x144" href="https://ecuestre.digital/wp-content/uploads/2021/01/cropped-Logo-Cuadrado-520-e1611039350171.png">
<link rel="apple-touch-icon-precomposed" sizes="114x114" href="https://ecuestre.digital/wp-content/uploads/2021/01/cropped-Logo-Cuadrado-520-e1611039350171.png">
<link rel="apple-touch-icon-precomposed" sizes="72x72" href="https://ecuestre.digital/wp-content/uploads/2021/01/cropped-Logo-Cuadrad


 <html class="js no-touch cssanimations csstransitions js csstransforms csstransforms3d csstransitions" lang="es-MX" style=""><head>
<meta charset="utf-8"/>
<!--[if IE]><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><![endif]-->
<title>Ecuestre Digital :   Show Info</title>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<meta content="" name="description"/>
<link href="https://ecuestre.digital/wp-content/uploads/2021/01/cropped-Logo-Cuadrado-520-e1611039350171.png" rel="shortcut icon"/>
<link href="https://ecuestre.digital/wp-content/uploads/2021/01/cropped-Logo-Cuadrado-520-e1611039350171.png" rel="apple-touch-icon-precomposed" sizes="144x144"/>
<link href="https://ecuestre.digital/wp-content/uploads/2021/01/cropped-Logo-Cuadrado-520-e1611039350171.png" rel="apple-touch-icon-precomposed" sizes="114x114"/>
<link href="https://ecuestre.digital/wp-content/uploads/2021/01/cropped-Logo-Cuadrado-520-e1611039350171.png" rel="apple-touch

In [32]:
links = soup.select('#custom-firebase-previous a')
suf_links_txt = [re.search('show-info[^"]*',str(i)).group(0) for i in links]
nombre = soup.select('#custom-firebase-previous p') 
info= [element.text.strip() for element in nombre]
ev = [info[i] for i in range(0,len(info),2)]
fecha = [info[i] for i in range(1,len(info),2)]
lugar = []
for w in ev:
    first, *middle, last = w.split()
    try:
        m = re.search(r'[A-Z]+[a-z]+$', last).group(0)
        lugar.append(m)
    except:
        m = last
        lugar.append(m)        
evento = [e.replace(l,"") for l,e in zip(lugar,ev)]
id_m = [re.findall('[0-9].*$',l)[0] for l in suf_links_txt]
id_main = [str(p[0:2])+'-'+str(re.search('.*[0-9]\s',s).group(0)).strip() for p,s in zip(lugar,fecha)]
df = pd.DataFrame(zip(id_m,evento,lugar,fecha,suf_links_txt))
df

,0,1,2,3,4
0,714,El Pulte,Guatemala,17/9/2021 - 19/9/2021,show-info?id=714
1,701,Club Hípico Yaguachi,Ecuador,17/9/2021 - 19/9/2021,show-info?id=701
2,712,Club Campestre Los Cortijos,Venezuela,17/9/2021 - 19/9/2021,show-info?id=712
3,660,"Major League Show JumpingTraverse City,",MI,17/9/2021 - 19/9/2021,show-info?id=660
4,716,Club Campestre De Cali,Colombia,16/9/2021 - 19/9/2021,show-info?id=716
5,709,Centro De Entrenamiento De Caballos Deportivos,Uruguay,11/9/2021 - 11/9/2021,show-info?id=709
6,710,Jockey Club De Caliente A.cBaja,California,10/9/2021 - 12/9/2021,show-info?id=710
7,704,Arrayanes Country Club,Ecuador,10/9/2021 - 12/9/2021,show-info?id=704
8,688,"Hípica Gdl, A.c.",Guadalajara,9/9/2021 - 12/9/2021,show-info?id=688
9,711,"Centro Ecuestre San Pedro, A.c.",Monterrey,9/9/2021 - 12/9/2021,show-info?id=711
